In [3]:
import numpy as np
import pandas as pd 
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\anaya\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
df = pd.read_csv(r'C:\Users\anaya\OneDrive\Desktop\Anay Masters\Intro to NLP\NLP_Group_7\FullyProcessedDataset.csv')

tokenized_corpus = [text.split() for text in df['Lemmatized_Lyrics']]

dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

lda_model = gensim.models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=35, random_state = 42)

song_topic_distribution = [lda_model[doc] for doc in corpus]


In [ ]:
def RecommendSongs(song_topic_distribution, user_input):

    user_song = (df.index[df['SName'] == user_input]).item()

#################################################################################

    def calculate_hellinger_distance(song_dist_1, song_dist_2):
        return hellinger(song_dist_1, song_dist_2)

    input_song_index = user_song

    input_song_dist = song_topic_distribution[input_song_index]

    hellinger_distances = []

    for i, song_dist in enumerate(song_topic_distribution):

        distance = calculate_hellinger_distance(input_song_dist, song_dist)
        hellinger_distances.append((distance))

    df['HDF'] = hellinger_distances

#################################################################################

    userGenre = df['Genres'].iloc[user_song]
    genreReducedDf = df.loc[df['Genres'] == userGenre]

#################################################################################

    indexList = genreReducedDf['Unnamed: 0.2'].tolist()
    newTopIndex = indexList[0]
    userSongIndexUpdated = user_song - newTopIndex
    analyzer = SentimentIntensityAnalyzer()

    def get_sentiment_score(lyrics):
        sentiment = analyzer.polarity_scores(lyrics)
        sentiment_score = sentiment['compound']
        return sentiment_score

    sentiment = get_sentiment_score(genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated])

    SSF_list = []
    for i in range(genreReducedDf.shape[0]):
        SSF_list.append(abs(genreReducedDf['Sentiment'].iloc[i] - sentiment))

    genreReducedDf['SSF'] = SSF_list

#################################################################################

    tfidf_vectorizer = TfidfVectorizer()
    CSF_list = []
    for i in range(genreReducedDf.shape[0]):
        lyrics_matrix = tfidf_vectorizer.fit_transform([genreReducedDf['Lemmatized_Lyrics'].iloc[userSongIndexUpdated], genreReducedDf['Lemmatized_Lyrics'].iloc[i]])
        lyrics_similarity = cosine_similarity(lyrics_matrix)
        CSF_list.append(1 - (lyrics_similarity[0][1]))
    genreReducedDf['CSF'] = CSF_list

#################################################################################

    genreReducedDf['SF'] = genreReducedDf['SSF'] + genreReducedDf['CSF'] + genreReducedDf['HDF']
    genreReducedDf = genreReducedDf.nsmallest(6,["SF"])
    genreReducedDf.drop(user_song, axis = 0, inplace = True)

#################################################################################

    RecommendationDF = genreReducedDf[['Artist', 'SName']]
    RecommendationDF.reset_index(inplace = True)
    RecommendationDF = RecommendationDF.drop('index', axis = 1)

#################################################################################

    return RecommendationDF